<a href="https://colab.research.google.com/github/HamzaMPSY/AI-Projects/blob/master/autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os 
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint 
from keras.utils import plot_model
from keras.models import model_from_json
import matplotlib.pyplot as plt

In [ ]:
INPUT_DIM = (160,160,3) # Image dimension
BATCH_SIZE = 64
Z_DIM = 128 # Dimension of the latent vector (z)

In [ ]:
cd drive/My\ Drive

/content/drive/My Drive


In [ ]:
images = np.load('utkfacesdata_160.npy')
images = np.transpose(images,(0,2,3,1))

In [ ]:
plt.imshow(images[3,...])

In [ ]:
def l2_normalize(x, axis=-1, epsilon=1e-10):
	"""
	function that normalize an np.array 
	"""
	output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
	return output

def preprocess(x):
	"""
	function to preprocess an image or array of images
	"""
	if x.ndim == 4:
	    axis = (1, 2, 3)
	    size = x[0].size
	elif x.ndim == 3:
	    axis = (0, 1, 2)
	    size = x.size
	else:
	    raise ValueError('Dimension should be 3 or 4')

	mean = np.mean(x, axis=axis, keepdims=True)
	std = np.std(x, axis=axis, keepdims=True)
	std_adj = np.maximum(std, 1.0/np.sqrt(size))
	y = (x - mean) / std_adj
	return y

In [ ]:
for i in range(images.shape[0]):
  images[i,...] = preprocess(images[i,...])

In [ ]:
# ENCODER
def build_encoder(input_dim, output_dim, conv_filters, use_batch_norm = True, use_dropout = True):
  global K
  K.clear_session()
  # Number of Conv layers
  n_layers = len(conv_filters)
  # Define model input
  encoder_input = Input(shape = input_dim, name = 'encoder_input')
  x = encoder_input
  # Add convolutional layers
  for i in range(n_layers):
      x = Conv2D(filters = conv_filters[i],kernel_size=3,strides=2,padding = 'same',name = 'encoder_conv_' + str(i))(x)
      if use_batch_norm:
        x = BatchNormalization()(x)
      x = LeakyReLU()(x)
      if use_dropout:
        x = Dropout(rate=0.25)(x)
  shape_before_flattening = K.int_shape(x)[1:] 
  x = Flatten()(x)
  encoder_output = Dense(Z_DIM)(x)
  return encoder_input, encoder_output ,shape_before_flattening, Model(encoder_input, encoder_output)

encoder_input, encoder_output,shape_before_flattening, encoder  = build_encoder(input_dim = INPUT_DIM,output_dim = Z_DIM,conv_filters = [32, 64])

In [ ]:
print(shape_before_flattening)
encoder.summary()

In [ ]:
def build_decoder(input_dim, shape_before_flattening, conv_filters):

  # Number of Conv layers
  n_layers = len(conv_filters)

  # Define model input
  decoder_input = Input(shape = (input_dim,) , name = 'decoder_input')

  # To get an exact mirror image of the encoder
  x = Dense(np.prod(shape_before_flattening))(decoder_input)
  x = Reshape(shape_before_flattening)(x)

  # Add convolutional layers
  for i in range(n_layers):
      x = Conv2DTranspose(filters = conv_filters[i], kernel_size = (3,3),strides=2,padding = 'same',name = 'decoder_conv_' + str(i))(x)
      # Adding a sigmoid layer at the end to restrict the outputs 
      # between 0 and 1
      if i < n_layers - 1:
        x = LeakyReLU()(x)
      else:
        x = Activation('sigmoid')(x)

  # Define model output
  decoder_output = x

  return decoder_input, decoder_output, Model(decoder_input, decoder_output)


decoder_input, decoder_output, decoder = build_decoder(input_dim = Z_DIM,
                                        shape_before_flattening = shape_before_flattening,
                                        conv_filters = [32,3])

In [ ]:
decoder.summary()
plot_model(decoder)

In [ ]:
# The input to the model will be the image fed to the encoder.
input = encoder_input

# Output will be the output of the decoder. The term - decoder(encoder_output) 
# combines the model by passing the encoder output to the input of the decoder.
output = decoder(encoder_output)

# Input to the combined model will be the input to the encoder.
# Output of the combined model will be the output of the decoder.
model = Model(input, output)

In [ ]:
model.summary()
plot_model(model)

In [ ]:
N_EPOCHS = 100
model.compile(optimizer='adadelta',loss='binary_crossentropy')
model.fit(images,images,epochs=N_EPOCHS)

In [ ]:
def loadModel():
	json_file = open('vae_model.json', 'r')
	loaded_model_json = json_file.read()
	json_file.close()
	loaded_model = model_from_json(loaded_model_json)
	# load weights into new model
	loaded_model.load_weights("vae_model.h5")
	print("Loaded model from disk")
	return loaded_model

vae_model = loadModel()


In [ ]:
import matplotlib.pyplot as plt

def plot_compare_vae(images=None):
  
  if images is None:
    example_batch = next(train_generator)
    example_batch = example_batch[0]
    images = example_batch[:10]

  n_to_show = images.shape[0]
  reconst_images = model.predict(images)

  fig = plt.figure(figsize=(15, 3))
  
  for i in range(n_to_show):
      img = images[i].squeeze()
      sub = fig.add_subplot(2, n_to_show, i+1)
      sub.axis('off')        
      sub.imshow(img)

  for i in range(n_to_show):
      img = reconst_images[i].squeeze()
      sub = fig.add_subplot(2, n_to_show, i+n_to_show+1)
      sub.axis('off')
      sub.imshow(img)  

plot_compare_vae(images[0:6]) 

In [ ]:
a = encoder.predict(images)

In [ ]:
# serialize classifier to JSON
classifier_json = encoder.to_json()
with open("encoder_mpsy.json", "w") as json_file:
    json_file.write(classifier_json)
# serialize weights to HDF5
encoder.save_weights("encoder_mpsy.h5")
print("Saved classifier to disk")

In [ ]:
print(np.linalg.norm(a[0] - a[-2]),np.linalg.norm(a[-1] - a[-2]))

In [ ]:
dist

In [ ]:
# serialize classifier to JSON
classifier_json = decoder.to_json()
with open("decodermpsy.json", "w") as json_file:
    json_file.write(classifier_json)
# serialize weights to HDF5
decoder.save_weights("decodermpsy.h5")
print("Saved classifier to disk")

In [ ]:
np.save("embeddings.npy",a)